In [50]:
import pandas as pd
from tqdm import tqdm


In [51]:
vad_df = pd.read_csv('NRC-VAD-Lexicon/NRC-VAD-Lexicon/NRC-VAD-Lexicon.txt',lineterminator='\n', delimiter='\t')


In [52]:
word='home'
word_series = vad_df[vad_df['Word']==word] 
if not word_series.empty:
    a=word_series['Valence'].to_numpy()
    b=word_series['Arousal'].to_numpy()
    d=word_series['Dominance'].to_numpy()

In [53]:
print(a)

[0.854]


In [54]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist as freq
import tqdm.notebook as tqdm

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/sahasp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sahasp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sahasp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [109]:
class vad_score():
    def __init__(self):
        self.vad = pd.read_csv('NRC-VAD-Lexicon/NRC-VAD-Lexicon/NRC-VAD-Lexicon.txt',lineterminator='\n', delimiter='\t')
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        
    def token_stemmer(self, inp_sentence, remove_stop):
        inp_sentence = inp_sentence.lower()
        token_list = self.tokenizer.tokenize(inp_sentence)
        if remove_stop:
            token_list = [word for word in token_list if word not in self.stop_words]
        lemma_list = [self.lemmatizer.lemmatize(word) for word in token_list]
        
        return lemma_list
    
    def get_vad_score(self, inp_sentence, remove_stop=False):
        token_list = self.token_stemmer(inp_sentence, remove_stop) or []
        fdist = dict(freq(token_list))
#         print(fdist)
        v_sum=0.0
        a_sum=0.0
        d_sum=0.0
        count=0.0
        
        for word in fdist.keys():
         
                word_series = self.vad[self.vad['Word']==word] 
                if not word_series.empty:
                    v_sum=word_series['Valence'].array[0] * fdist[word]
                    a_sum=word_series['Arousal'].array[0] * fdist[word]
                    d_sum=word_series['Dominance'].array[0]  * fdist[word]
                    count += fdist[word]
           
           
        count = max(count, 1)
        return (v_sum/count, a_sum/count, d_sum/count)
    
           

In [110]:

l = vad_score()
print(l.token_stemmer('I am a man with hello home potential and needs', True))
print(l.get_vad_score('I am a man hearts heart heartily with hello and i is a man with home potential and needs' ))

print(l.get_vad_score('I am a man with hello home potential and needs'.lower(),True ))

print(l.token_stemmer('He is and was and always will be mine i loved her. She was so pretty and lovely what can i say I really love her', False))

print(l.get_vad_score('Sahas is god '))

['man', 'hello', 'home', 'potential', 'need']
(0.06375, 0.07175, 0.025)
(0.1275, 0.1435, 0.05)
['he', 'is', 'and', 'wa', 'and', 'always', 'will', 'be', 'mine', 'i', 'loved', 'her', 'she', 'wa', 'so', 'pretty', 'and', 'lovely', 'what', 'can', 'i', 'say', 'i', 'really', 'love', 'her']
(0.758, 0.518, 0.8859999999999999)


In [111]:
import pandas as pd

In [112]:
clean_movies_df = pd.read_csv('clean/updated_movies_features_clean.csv', lineterminator='\n')

In [113]:
clean_movies_df[clean_movies_df['overview'].isna()]

,Unnamed: 0,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge


In [114]:
l.get_vad_score(clean_movies_df['overview'][0],True)

(0.009000000000000001, 0.011159090909090908, 0.013886363636363636)

In [115]:

def vad_analyis(row):
#     print(row.name)
    v,a,d = vad.get_vad_score(row.overview, remove_stop=True)
    data = {'tmdbId':row.id, 'valence': v, 'arousal': a, 'dominance': d}
    return pd.Series(data)

In [116]:
empty_overview_df = clean_movies_df[clean_movies_df['overview'].isnull()]

In [117]:
empty_overview_df

,Unnamed: 0,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge


In [118]:
from tqdm import tqdm
vad = vad_score()
tqdm.pandas()

vad_df = pd.DataFrame(columns=['tmdbId', 'valence', 'arousal', 'dominance'])
# a = clean_movies_df.apply(vad_analyis, axis=1)

a = clean_movies_df.progress_apply(vad_analyis, axis=1)


/home/sahasp/.local/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 62000/62000 [1:08:09<00:00, 15.16it/s]


In [120]:
a.head()

,tmdbId,valence,arousal,dominance
0,525153.0,0.009000,0.011159,0.013886
1,503475.0,0.015941,0.014412,0.015912
2,581579.0,0.004035,0.009789,0.008860
3,525936.0,0.018486,0.006757,0.014270
4,484245.0,0.035714,0.010714,0.029238


In [99]:
a

,tmdbId,valence,arousal,dominance
0,525153.0,0.009000,0.011159,0.013886
1,503475.0,0.015941,0.014412,0.015912
2,581579.0,0.004035,0.009789,0.008860
3,525936.0,0.018486,0.006757,0.014270
4,484245.0,0.035714,0.010714,0.029238
5,470646.0,0.014488,0.011756,0.015561
6,575751.0,0.013455,0.009758,0.011909
7,566972.0,0.078857,0.037143,0.066286
8,572997.0,0.000000,0.000000,0.000000
9,559489.0,0.132000,0.060833,0.088000


In [121]:
a.to_csv('clean/updated_vad.csv')


In [122]:
vad_df = pd.read_csv('clean/updated_vad.csv')

In [123]:
vad_df

,Unnamed: 0,tmdbId,valence,arousal,dominance
0,0,525153.0,0.009000,0.011159,0.013886
1,1,503475.0,0.015941,0.014412,0.015912
2,2,581579.0,0.004035,0.009789,0.008860
3,3,525936.0,0.018486,0.006757,0.014270
4,4,484245.0,0.035714,0.010714,0.029238
...,...,...,...,...,...
61995,61995,439050.0,0.216250,0.125000,0.144500
61996,61996,111109.0,0.075714,0.067429,0.079714
61997,61997,67758.0,0.024667,0.065667,0.047250
61998,61998,227506.0,0.015764,0.010291,0.010709


In [ ]:

vad_df = pd.DataFrame(columns=['A','B'])
vad_df = vad_df.append({'A':1,'B':2}, ignore_index=True)


vad_df = vad_df.append({'A':2,'B':2}, ignore_index=True)

vad_df = vad_df.append({'A':3,'B':2}, ignore_index=True)

In [112]:
vad_df

,A,B
0,1,2
1,2,2
2,3,2


In [12]:
clean_movies_df

,Unnamed: 0,index,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge
0,0,0,False,525153,tt0094878,en,"During the Second World War, tens of thousands...",1.201000,/emSTzEr2qdo3lV14ZQ3uIYsqHJS.jpg,101.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Shadows,0.0,0.0,left_only
1,1,1,False,503475,NaN,en,Anthology of Aardman Animation short films rel...,2.272000,/tWkYclnnChwoMMBFECsEjd5hb77.jpg,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Wallace & Gromit: The Best of Aardman Animation,7.5,2.0,left_only
2,2,2,False,581579,tt0116992,en,Following the life of Marlene Dietrich through...,0.600000,/faHxYNlQ7RGDtqmaNBpFf0HKozg.jpg,52.0,[],Marlene Dietrich: Shadows and Light,0.0,0.0,left_only
3,3,3,False,525936,tt0112746,en,Lonely Kansas housewife Karen Webb despondent ...,0.841000,NaN,81.0,[],The Crude Oasis,0.0,0.0,left_only
4,4,4,False,484245,tt0116536,en,Back in the stone aged we all went to San Fran...,0.655000,/p5SC0B48GKtwOMLVK0qdY4p164F.jpg,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",The Hippie Revolution,0.0,0.0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,64613,67317,False,439050,tt6209470,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Subdue,4.0,1.0,right_only
61996,64614,67318,False,111109,tt2028550,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,9.0,3.0,right_only
61997,64615,67319,False,67758,tt0303758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Betrayal,3.8,6.0,right_only
61998,64616,67320,False,227506,tt0008536,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,87.0,[],Satan Triumphant,0.0,0.0,right_only


In [32]:
clean_movies_df['overview'][clean_movies_df['id']==525153] ='b'

<ipython-input-32-5fe9828338c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_movies_df['overview'][clean_movies_df['id']==525153] ='b'


In [33]:
clean_movies_df

,Unnamed: 0,index,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge
0,0,0,False,525153,tt0094878,en,b,1.201000,/emSTzEr2qdo3lV14ZQ3uIYsqHJS.jpg,101.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Shadows,0.0,0.0,left_only
1,1,1,False,503475,NaN,en,Anthology of Aardman Animation short films rel...,2.272000,/tWkYclnnChwoMMBFECsEjd5hb77.jpg,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Wallace & Gromit: The Best of Aardman Animation,7.5,2.0,left_only
2,2,2,False,581579,tt0116992,en,Following the life of Marlene Dietrich through...,0.600000,/faHxYNlQ7RGDtqmaNBpFf0HKozg.jpg,52.0,[],Marlene Dietrich: Shadows and Light,0.0,0.0,left_only
3,3,3,False,525936,tt0112746,en,Lonely Kansas housewife Karen Webb despondent ...,0.841000,NaN,81.0,[],The Crude Oasis,0.0,0.0,left_only
4,4,4,False,484245,tt0116536,en,Back in the stone aged we all went to San Fran...,0.655000,/p5SC0B48GKtwOMLVK0qdY4p164F.jpg,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",The Hippie Revolution,0.0,0.0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,64613,67317,False,439050,tt6209470,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Subdue,4.0,1.0,right_only
61996,64614,67318,False,111109,tt2028550,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,9.0,3.0,right_only
61997,64615,67319,False,67758,tt0303758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Betrayal,3.8,6.0,right_only
61998,64616,67320,False,227506,tt0008536,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,87.0,[],Satan Triumphant,0.0,0.0,right_only


In [21]:
a = clean_movies_df[clean_movies_df['overview'].isnull()]
imdb_id_list = a[a['imdb_id'].notnull()]['imdb_id'].values.tolist()
tmdb_id_list = a[a['imdb_id'].isnull()]['id'].values

In [22]:
print(len(imdb_id_list))
# print(len(tmdb_id))
# tmdb_id
imdb_id_list

350


['tt0092281',
 'tt0138674',
 'tt0141762',
 'tt1132471',
 'tt0161420',
 'tt0211052',
 'tt0080729',
 'tt0100815',
 'tt0084959',
 'tt1514834',
 'tt0078981',
 'tt0050101',
 'tt0047896',
 'tt0051000',
 'tt0062242',
 'tt0061090',
 'tt0182379',
 'tt0157360',
 'tt0103970',
 'tt0094471',
 'tt0089542',
 'tt0094836',
 'tt0079958',
 'tt0036483',
 'tt0274874',
 'tt0168649',
 'tt0077748',
 'tt0020835',
 'tt1144794',
 'tt0091437',
 'tt0055574',
 'tt0056638',
 'tt0125157',
 'tt0060336',
 'tt0103194',
 'tt0348601',
 'tt6822272',
 'tt2614370',
 'tt0261318',
 'tt0165138',
 'tt1259770',
 'tt1846499',
 'tt0330951',
 'tt1931379',
 'tt0308051',
 'tt1085807',
 'tt0311986',
 'tt1085882',
 'tt1085824',
 'tt2435106',
 'tt5352872',
 'tt1839450',
 'tt1483361',
 'tt5435740',
 'tt0294997',
 'tt0832879',
 'tt0376114',
 'tt1485750',
 'tt2670824',
 'tt1776283',
 'tt0090599',
 'tt0963997',
 'tt5123264',
 'tt0499204',
 'tt0202639',
 'tt0179227',
 'tt0082710',
 'tt3142960',
 'tt1927001',
 'tt0168679',
 'tt3999186',
 'tt01

In [25]:
import requests
from tqdm import tqdm
for id in tqdm(imdb_id_list):
    response = requests.get("https://www.omdbapi.com/?i="+str(id)+"&apikey=e4c29baa")
    if(response.status_code==200):
        json = response.json()
        clean_movies_df['overview'][clean_movies_df['imdb_id']==id] =json['Plot']

  0%|          | 0/350 [00:00<?, ?it/s]<ipython-input-25-75cfc22c490b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_movies_df['overview'][clean_movies_df['imdb_id']==id] =json['Plot']
100%|██████████| 350/350 [00:45<00:00,  7.64it/s]


In [64]:
from tqdm import tqdm
for i in tqdm([1,2,3]):
    print(i)

100%|██████████| 3/3 [00:00<00:00, 12240.19it/s]

1
2
3


In [49]:
a = clean_movies_df[clean_movies_df['overview'].isnull()]
imdb_id_list = a[a['imdb_id'].notnull()]['imdb_id'].values.tolist()
tmdb_id_list = a[a['imdb_id'].isnull()]['id'].values
len(tmdb_id_list)

3

In [80]:
clean_movies_df.to_csv('clean/updated_movies_features_clean.csv')

In [31]:
clean_movies_df[6:10]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,index,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge
6,6,6,6,6,6,False,575751,tt0111357,en,Andie Norman (Kim Wayans) is a young publisher...,0.600,/3tNISfteDbGI09O5HZnedzOFekb.jpg,87.0,[],Talking About Sex,0.0,0.0,left_only
7,7,7,7,7,7,False,566972,tt0113328,nl,Old entertainer gets a last chance to play a s...,0.620,/k36bQGCz3YS6yG8VrnmCyGtwFqs.jpg,120.0,[],Last Call,7.0,1.0,left_only
8,8,8,8,8,8,False,572997,tt0092281,pl,Zaproszenie,0.737,/zWOoJagc3FvGJZdXdzhQf56gFsv.jpg,92.0,[],Zaproszenie,0.0,0.0,left_only
9,9,9,9,9,9,False,559489,tt0113031,en,A career thief forms a partnership with his re...,0.600,NaN,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Farmer & Chase,0.0,0.0,left_only


In [79]:
clean_movies_df[:10]

,Unnamed: 0,index,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge
0,0,0,False,525153,tt0094878,en,"During the Second World War, tens of thousands...",1.201,/emSTzEr2qdo3lV14ZQ3uIYsqHJS.jpg,101.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Shadows,0.0,0.0,left_only
1,1,1,False,503475,NaN,en,Anthology of Aardman Animation short films rel...,2.272,/tWkYclnnChwoMMBFECsEjd5hb77.jpg,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Wallace & Gromit: The Best of Aardman Animation,7.5,2.0,left_only
2,2,2,False,581579,tt0116992,en,Following the life of Marlene Dietrich through...,0.600,/faHxYNlQ7RGDtqmaNBpFf0HKozg.jpg,52.0,[],Marlene Dietrich: Shadows and Light,0.0,0.0,left_only
3,3,3,False,525936,tt0112746,en,Lonely Kansas housewife Karen Webb despondent ...,0.841,NaN,81.0,[],The Crude Oasis,0.0,0.0,left_only
4,4,4,False,484245,tt0116536,en,Back in the stone aged we all went to San Fran...,0.655,/p5SC0B48GKtwOMLVK0qdY4p164F.jpg,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",The Hippie Revolution,0.0,0.0,left_only
5,5,5,False,470646,tt0114147,en,Funny yet incisive look at the PANTEX Nuclear ...,0.625,/s8ys1qnwM8VU6jIPlrRoiBz40RV.jpg,73.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Plutonium Circus,6.0,1.0,left_only
6,6,6,False,575751,tt0111357,en,Andie Norman (Kim Wayans) is a young publisher...,0.600,/3tNISfteDbGI09O5HZnedzOFekb.jpg,87.0,[],Talking About Sex,0.0,0.0,left_only
7,7,7,False,566972,tt0113328,nl,Old entertainer gets a last chance to play a s...,0.620,/k36bQGCz3YS6yG8VrnmCyGtwFqs.jpg,120.0,[],Last Call,7.0,1.0,left_only
8,8,8,False,572997,tt0092281,pl,N/A,0.737,/zWOoJagc3FvGJZdXdzhQf56gFsv.jpg,92.0,[],Zaproszenie,0.0,0.0,left_only
9,9,9,False,559489,tt0113031,en,A career thief forms a partnership with his re...,0.600,NaN,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Farmer & Chase,0.0,0.0,left_only


In [60]:
for id in tqdm(tmdb_id_list):
        print(clean_movies_df['title'][clean_movies_df['id']==id])
        clean_movies_df['overview'][clean_movies_df['id']==id] = clean_movies_df['title'][clean_movies_df['id']==id]

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-60-606aa840cdd0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_movies_df['overview'][clean_movies_df['id']==id] = clean_movies_df['title'][clean_movies_df['id']==id]
<ipython-input-60-606aa840cdd0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_movies_df['overview'][clean_movies_df['id']==id] = clean_movies_df['title'][clean_movies_df['id']==id]
<ipython-input-60-606aa840cdd0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

3768    Contergan
Name: title, dtype: object
32511    Show
Name: title, dtype: object
58394    Le passe-muraille
Name: title, dtype: object


In [81]:
clean_movies_df = pd.read_csv('clean/updated_movies_features_clean.csv', lineterminator='\n')

In [85]:
clean_movies_df[clean_movies_df['overview'].isnull()]

,Unnamed: 0,adult,id,imdb_id,original_language,overview,popularity,poster_path,runtime,spoken_languages,title,vote_average,vote_count,_merge


In [78]:
# clean_movies_df.drop(clean_movies_df.columns[0], axis=1, inplace=True)

In [112]:
vad_df = pd.read_csv('clean/vad.csv')

In [129]:
vad_df[vad_df['valenc'].isnull()]

,Unnamed: 0,tmdbId,valence,arousal,dominance
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
61995,NaN,NaN,NaN,NaN,NaN
61996,NaN,NaN,NaN,NaN,NaN
61997,NaN,NaN,NaN,NaN,NaN
61998,NaN,NaN,NaN,NaN,NaN


In [13]:
import pandas as pd


In [14]:
word_df =pd.read_csv('clean/Vectorized.csv')

In [15]:
word_df

,Unnamed: 0,tmdbId,vector
0,0,525153,[ 0.06721339 0.04038396 0.06326807 0.036057...
1,1,503475,[ 3.28043275e-02 3.64426435e-03 -1.06155654e-...
2,2,581579,[ 4.72832137e-02 4.26037668e-02 -8.98567287e-...
3,3,525936,[ 0.02534703 0.02237873 -0.06309042 0.061940...
4,4,484245,[ 0.04845082 0.02994283 0.01425633 0.012097...
...,...,...,...
61995,61995,439050,[ 0.12194061 0.16162109 -0.14575195 -0.047004...
61996,61996,111109,[ 1.07645671e-01 5.30327691e-02 -1.20307075e-...
61997,61997,67758,[ 5.63313802e-02 -3.18766276e-02 2.28352865e-...
61998,61998,227506,[ 0.03414848 0.02131977 0.02590653 0.022190...


In [43]:
len(word_df['vector'][0])

3645

In [21]:
df = pd.DataFrame({'a': [[1,2,3,4]]})

In [42]:
len(df['a'][0])

4

In [48]:
type(df['a'])

pandas.core.series.Series

In [47]:
type(df['a'][0])

list

In [34]:
series = pd.Series({'a': [1,2,3,4]})

In [44]:
len(series[0])

4

In [38]:
series[0][0]

1